# Data engineering Healthcare Project
---
### Data Analysis 

In this notebook, we analyze the provided datasets and compare three database systems.
We also motivate our final database choice based on the structure and properties of the data.

### 1. Importing Our Tools (Libraries) & Loading Data
We will use pandas for data manipulation, and matplotlib and seaborn for creating charts. This is the standard and most effective toolkit for this kind of analysis in Python. After importing, we load our six pre-cleaned CSV files into pandas DataFrames.

* import pandas as pd: We use pandas to work with data tables (like spreadsheets). It lets us load our data from CSV files into something called a DataFrame. Think of a DataFrame as a smart table where we can easily sort, filter, and work with our data.
* Why pandas? Why not just use Python's built-in lists or dictionaries? While we could use basic Python to handle the data, it would be much harder. We would have to write a lot more code to do simple things like calculating the average of a column. pandas is the standard tool for data analysis in Python because it's powerful and easy to use for table-like data.
* import numpy as np: numpy is a library for working with numbers and math. pandas actually uses numpy behind the scenes. We import it directly because it's a good habit and sometimes we need it for special math operations.
* import matplotlib.pyplot as plt and import seaborn as sns: These two libraries are used for making charts and graphs (visualizations). Pictures help us understand the data much better than just looking at numbers.
* Why use two plotting libraries? matplotlib is the original, foundational plotting library. It can make almost any chart, but it can sometimes take a lot of code to make it look good. seaborn is built on top of matplotlib and makes it much easier to create beautiful and common statistical plots (like bar charts and heatmaps) with less code. We use them together to get the best of both worlds.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

### 2. Loading the Data Files
Now we use the pandas library, which we called pd, to load our six CSV files. Each file contains a different piece of the healthcare puzzle. We give each one a unique name so we can work with it separately.

Our teammate already cleaned the data a little bit with general functions

In [29]:
observations = pd.read_csv("../cleaned_data/observations_cleaned.csv")
patients = pd.read_csv("../cleaned_data/patients_cleaned.csv")
procedures = pd.read_csv("../cleaned_data/procedures_cleaned.csv")

diagnoses = pd.read_csv("../cleaned_data/diagnoses_cleaned.csv")
encounters = pd.read_csv('../cleaned_data/encounters_cleaned.csv')
medications = pd.read_csv('../cleaned_data/medications_cleaned.csv')

### 3. Making a Smart Plan: The Reusable Analysis Function
We have six different data files to check. We could copy and paste our analysis code six times, but that's a bad idea.
* The wrong way (the alternative): If we copied and pasted the code, our notebook would be very long and repetitive. Even worse, if we found a mistake or wanted to add a new chart, we would have to fix it in all six places! It's easy to make a mistake this way.
* The right way (our implementation): We create a single, reusable function called analyze_dataframe. Think of it like a recipe. We write the steps for our analysis once inside this function. Then, for each data file, we just "call" the function. This is cleaner, safer, and much easier to manage.

#### What's Inside Our "Recipe"?
Our function will automatically perform a standard check-up on any DataFrame we give it:
1. .shape, .head(), .info(): These commands give us a quick first look. We see how many rows and columns there are, what the first few rows look like, and what kind of data is in each column (e.g., numbers, text, dates).
2. .isnull().sum(): This checks for missing data. Missing values can cause errors, so we need to know where they are.
3. .describe(include='all'): This gives us statistics. For number columns, it shows the average, min, max, etc. By adding include='all', we tell it to also give us information about text columns, like how many unique text values there are.
4. Visualizations (Charts): We create a few standard charts to help us "see" the data.
* Histograms for numbers: This is the best way to see the distribution or "shape" of numerical data. It shows us if most values are low, high, or spread out evenly.
* Bar charts for text: This is perfect for counting how many times each item in a category appears.
* Why not a pie chart? Pie charts can be hard to read if there are more than two or three categories. It's difficult for our eyes to compare the size of slices. Bar charts are almost always easier to understand and compare.
* Safety Check (if df[col].nunique() < 20): We only create bar charts for text columns with fewer than 20 unique categories. Why? Imagine trying to create a bar chart for a "patient_name" column with 1000 names. It would be a giant, unreadable mess. This check prevents that and keeps our analysis useful.


In [ ]:
def analyze_dataframe(df, filename):
    """
    Performs and prints a full analysis for a given DataFrame,
    including data visualizations.
    """
    print("-" * 50)
    print(f"ANALYSIS FOR: {filename}")
    print("-" * 50)
    
    # Shape
    print(f"\n[INFO] Shape of the dataset (rows, columns): {df.shape}\n")
    
    # Head
    print("[INFO] First 5 rows of the dataset:")
    print(df.head())
    print("\n" + "-" * 30 + "\n")
    
    # Info
    print("[INFO] Technical Information (Data Types and Non-Null Counts):")
    df.info()
    print("\n" + "-" * 30 + "\n")

    # Missing Values
    print("[INFO] Count of Missing (Null) Values per Column:")
    missing_values = df.isnull().sum()
    if missing_values.sum() == 0:
        print("No missing values found.")
    else:
        print(missing_values[missing_values > 0])
    print("\n" + "-" * 30 + "\n")

    # Descriptive Statistics
    print("[INFO] Descriptive Statistics for All Columns:")
    print(df.describe(include='all'))
    print("\n\n")
    
    # VISUALIZATIONS
    print("-" * 50)
    print(f"VISUALIZATIONS FOR: {filename}")
    print("-" * 50)

    # Missing Value Heatmap
    if df.isnull().sum().sum() > 0:
        plt.figure(figsize=(12, 6))
        sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
        plt.title(f'Missing Value Heatmap for {filename}')
        plt.show()
    
    # Histograms for Numerical Columns
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    if len(numeric_cols) > 0:
        print(f"\n[VISUAL] Histograms for Numerical Columns in {filename}:")
        df[numeric_cols].hist(bins=15, figsize=(15, 10), layout=(-1, 3))
        plt.tight_layout()
        plt.show()
    
    # Count Plots for Categorical Columns
    # We only plot categorical columns with a reasonable number of unique values (< 20)
    categorical_cols = df.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        print(f"\n[VISUAL] Count Plots for Key Categorical Columns in {filename}:")
        for col in categorical_cols:
            if df[col].nunique() < 20:
                plt.figure(figsize=(10, 5))
                sns.countplot(y=col, data=df, order=df[col].value_counts().index)
                plt.title(f'Count of each category in: {col}')
                plt.xlabel('Count')
                plt.ylabel(col)
                plt.tight_layout()
                plt.show()
    
    print("\n\n")

 ---
### 2. Dataset-Specific Analysis
Patients Dataset
Summary: This is the master table for patient demographic data. It contains Personally Identifiable Information (PII) like names, addresses, and birth dates, which is a major consideration for data security (GDPR). The data is high quality with no missing values. The date_of_birth column is stored as a string (object) and will require transformation into a proper date type for age calculations. The patient_id column serves as the primary key to link to all other datasets.

In [ ]:
analyze_dataframe(observations, 'Oberservations')

#### Deeper Analysis & Commentary: Observations
* Structure and Keys: With 886 rows and 9 columns, this is a "tall" table, which is typical for transactional or event-based data like lab results. The observation_id column is unique for every single row, which makes it the Primary Key for this table. This means we can use this ID to refer to a specific, unique observation. patient_id and encounter_id are Foreign Keys, linking each observation back to a patient and a specific visit.
* Data Types - A Critical Finding: The observation_datetime column is stored as an object (text), not a proper date type. This is a crucial finding. For our database, we must store this as a DATETIME or TIMESTAMP type. This allows for correct sorting, filtering by date ranges, and calculating time differences, which is impossible with text.
* Missing Value Pattern: This is very insightful. units and value_numeric are both missing 25 values. Meanwhile, value_text contains exactly 25 non-null values. This tells a clear story: an observation is either a number (with units) OR it's a text value (like "Positive" or "Negative"). This is actually a good, clean data structure. Our database design should probably have a value_numeric column and a value_text column to reflect this.
* Content Insights:
* The dataset contains results for only 9 unique types of observations. The most common are lab tests like 'Creatinine' and 'Hemoglobin A1c', as well as blood pressure readings.
* These observations belong to 150 unique patients, which matches the total number of patients in our patients table.

In [ ]:
analyze_dataframe(patients, 'Patients')

#### Deeper Analysis & Commentary: Patients
* Structure and Keys: This is our central "master" table for patients. With 150 rows and 10 columns, each row represents one unique person. The patient_id column has 150 unique values, confirming it is the Primary Key. All other tables will link to this one using patient_id.
* Data Quality: The data quality is excellent, with no missing values. This is great, but in a real-world scenario, this is rare.
* Data Types - Important for Engineering:
* Just like in the previous table, date_of_birth is an object (text). This MUST be converted to a DATE type in our final database. This is essential for correctly calculating patient ages.
* zip_code is stored as an int64 (a number). This is generally acceptable, but it's often better practice to store codes like this as text (VARCHAR in a database). This prevents issues like dropping leading zeros (e.g., '07039' becoming '7039') and makes it clear that we shouldn't perform mathematical operations (like calculating the average) on them.
* Content Insights:
* The data contains sensitive, Personally Identifiable Information (PII) like names, addresses, and phone numbers. This has huge implications for our data engineering. The database and any applications built on it must have strong security measures, including encryption and access controls, to comply with privacy laws like GDPR and HIPAA.
* The gender split is almost 50/50 (79 Male vs. 71 Female).
* The most common last name is 'Smith', which is a common feature of synthetically generated data.

In [ ]:
analyze_dataframe(procedures, 'Procedures')

#### Deeper Analysis & Commentary: Procedures
* Structure and Scope: This is a very small table with only 54 records. This tells us that either procedures are rare events for this patient group, or the dataset is limited in scope.
* Data Types: Once again, date_performed is an object (text) and must be converted to a DATE or DATETIME type in the database.
* Content Insights:
* The most striking observation is that there are only 2 unique procedures in the entire dataset: "Influenza virus vaccine, quadrivalent" and "Ophthalmological examination...". This is a very limited set. This means we cannot use this data to analyze a wide range of surgical or medical procedures.
* 50 unique patients are represented, with one patient having had 2 procedures.
* The procedure_id is the Primary Key for this table as it's unique for every row.

In [ ]:
analyze_dataframe(diagnoses, 'Diagnoses')

#### Deeper Analysis & Commentary: Diagnoses
* Structure: With 221 records for 150 patients, it's clear that patients can have more than one diagnosis, which is expected.
* Data Types: date_recorded is an object and should be converted to a DATE type.
* Content Insights:
* Similar to procedures, the scope is very limited. There are only 5 unique diagnoses.
* The most common diagnosis is 'Essential (primary) hypertension' (high blood pressure), appearing 65 times. The next most common is 'Type 2 diabetes mellitus'. This gives us a very clear picture of the primary health concerns of this patient population. This aligns with the 'Metformin' medication we might see later.
* diagnosis_id is the Primary Key. encounter_id is also unique for every row, suggesting that in this dataset, only one diagnosis is recorded per encounter.

In [ ]:
analyze_dataframe(encounters, 'Encounters')

#### Deeper Analysis & Commentary: Encounters
* Structure: This table acts as the "glue," with 291 unique visits linking patients to physicians and other events.
* Data Types: admission_date and discharge_date are both text (object) and need to be converted to DATETIME types. This would allow us to calculate the duration of a visit.
* The Most Important Finding: The visit_type column is the most critical insight from this table. It has only one unique value: 'Outpatient'. This is a major constraint on our dataset. It means we have no data on hospitalizations, emergency room visits, or telehealth appointments. Any analysis or system we build must acknowledge that it is based purely on outpatient data.
* Content Insights:
* The dataset covers all 150 patients.
* There are 108 unique attending_physician_ids, indicating a fairly large number of doctors are involved.

In [ ]:
analyze_dataframe(medications, 'Medications')

#### Deeper Analysis & Commentary: Medications
* Structure: 160 medication orders for 126 unique patients. This means not all patients in the dataset are on medication.
* Missing Values - A Key Story:
* end_date is missing for 120 of the 160 records (75%). This is not a data quality problem; it's a feature. It tells us that these are likely active, ongoing prescriptions. A null end_date means the patient is still supposed to be taking the medication. This is a critical business rule to build into our database logic.
* dosage is missing for 15 records. This, unlike the end_date, is likely a data quality issue that would need to be addressed in a real project.
* Data Types: start_date and end_date are text (object) and must be converted to DATE types.
* Content Insights:
* There are only 5 unique drugs. The most common is 'Metformin HCl 500mg tablet', a drug for diabetes. This, combined with our findings from the diagnoses table, confirms that diabetes and hypertension are the primary conditions managed in this dataset.
* All medications are taken 'Oral'-ly. There are no injections or other routes of administration. This further simplifies the nature of our data.